# Exp T1d: Lifetime Measurement (Ordered → Collapse, High Noise)

## 目的
高ノイズ環境(η=0.8)でのOrdered→Collapse寿命を測定。

## 実験設計
- **初期化**: λ=0.30で30 epoch学習（高ノイズでもordered確立）
- **Target λ**: 0.44（R3aでλ_↑≈0.44と判明した転移点）
- **η**: 0.8（HIGH NOISE）
- **観測**: 崩壊イベント（error≥0.50が5 epoch継続）までの時間
- **打ち切り**: +150 epoch（censored）
- **Seeds**: 50

## 推定時間
~50 × 10 min ≈ **8h**

## 科学的目標
- 高ノイズでの転移点近傍での寿命分布
- η=0.4との比較（ノイズ依存性）
- PRX Figure: η依存性の定量化

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os, glob, json, time
from datetime import datetime

EXP_NAME = 'exp_T1_lifetime_ordered'
NOTEBOOK_ID = 'T1d'
BASE_DIR = '/content/drive/MyDrive/dual-gradient-learning/Paper-A'

existing = glob.glob(f'{BASE_DIR}/{EXP_NAME}_*')
if existing:
    SAVE_DIR = sorted(existing)[-1]
    print(f'🔄 Resuming: {SAVE_DIR}')
else:
    TIMESTAMP = datetime.now().strftime('%Y%m%d_%H%M%S')
    SAVE_DIR = f'{BASE_DIR}/{EXP_NAME}_{TIMESTAMP}'
    os.makedirs(SAVE_DIR, exist_ok=True)
    print(f'🆕 New: {SAVE_DIR}')

os.makedirs(f'{SAVE_DIR}/figures', exist_ok=True)
os.makedirs(f'{SAVE_DIR}/checkpoints', exist_ok=True)
print(f'Notebook: {NOTEBOOK_ID} (O→C lifetime, λ=0.44, η=0.8 HIGH NOISE)')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils import parameters_to_vector
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name()}')

In [ ]:
# Experiment parameters
BATCH_SIZE = 256
NUM_WORKERS = 4
LR = 0.1
K = 16

# Lifetime experiment specific - HIGH NOISE
NOISE_RATE = 0.8         # ← HIGH NOISE!
INIT_LAMBDA = 0.30       # Lower λ needed for ordered init at high noise
INIT_EPOCHS = 30
TARGET_LAMBDA = 0.44     # ← R3a: λ_↑ ≈ 0.44 for η=0.8
MAX_OBSERVE_EPOCHS = 150
COLLAPSE_THRESHOLD = 0.50  # Higher threshold for high noise
COLLAPSE_PERSIST = 5

N_SEEDS = 50

print(f'⚠️ HIGH NOISE: η = {NOISE_RATE}')
print(f'Init: λ={INIT_LAMBDA} for {INIT_EPOCHS} epochs')
print(f'Target: λ={TARGET_LAMBDA}, max {MAX_OBSERVE_EPOCHS} epochs')
print(f'Collapse: error≥{COLLAPSE_THRESHOLD} for {COLLAPSE_PERSIST} epochs')
print(f'Seeds: {N_SEEDS}')

In [ ]:
def get_resnet18():
    model = resnet18(weights=None, num_classes=10)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    model.maxpool = nn.Identity()
    return model

class IndexedDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
    def __getitem__(self, idx):
        img, label = self.dataset[idx]
        return img, label, idx
    def __len__(self):
        return len(self.dataset)

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def inject_label_noise(labels, noise_rate, seed):
    np.random.seed(seed)
    noisy = labels.copy()
    n_noisy = int(noise_rate * len(labels))
    idx = np.random.choice(len(labels), n_noisy, replace=False)
    for i in idx:
        noisy[i] = np.random.choice([l for l in range(10) if l != labels[i]])
    return noisy

def load_cifar10():
    tr = transforms.Compose([transforms.RandomCrop(32, padding=4), transforms.RandomHorizontalFlip(),
                             transforms.ToTensor(), transforms.Normalize((0.4914,0.4822,0.4465),(0.2023,0.1994,0.2010))])
    te = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914,0.4822,0.4465),(0.2023,0.1994,0.2010))])
    return torchvision.datasets.CIFAR10('./data', True, tr, download=True), torchvision.datasets.CIFAR10('./data', False, te, download=True)

def evaluate(model, loader):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            correct += (model(x).argmax(1) == y).sum().item()
            total += y.size(0)
    return correct / total

In [ ]:
def train_one_epoch(model, train_loader, opt, sched, clean_t, noisy_t, lam, cached_gv_ref):
    crit = nn.CrossEntropyLoss()
    model.train()
    step = cached_gv_ref['step']
    cached_gv = cached_gv_ref['gv']
    cos_list = []
    
    for x, _, idx in train_loader:
        x, idx = x.to(device), idx.to(device)
        bn, bc = noisy_t[idx], clean_t[idx]
        
        opt.zero_grad()
        loss_s = crit(model(x), bn)
        loss_s.backward(retain_graph=True)
        gs = parameters_to_vector([p.grad for p in model.parameters()]).clone()
        
        if step % K == 0 or cached_gv is None:
            opt.zero_grad()
            loss_v = crit(model(x), bc)
            loss_v.backward()
            cached_gv = parameters_to_vector([p.grad for p in model.parameters()]).clone()
        
        gs_n = gs / (gs.norm() + 1e-12)
        gv_n = cached_gv / (cached_gv.norm() + 1e-12)
        cos_list.append((gs_n @ gv_n).item())
        
        g_mix = (1 - lam) * gs_n + lam * gv_n
        opt.zero_grad()
        i = 0
        for p in model.parameters():
            n = p.numel()
            p.grad = g_mix[i:i+n].view(p.shape).clone()
            i += n
        opt.step()
        step += 1
    
    if sched is not None:
        sched.step()
    
    cached_gv_ref['step'] = step
    cached_gv_ref['gv'] = cached_gv
    return np.mean(cos_list)

In [ ]:
def run_lifetime_experiment(seed, train_loader, test_loader, clean_labels, noisy_labels):
    clean_t = torch.tensor(clean_labels, device=device)
    noisy_t = torch.tensor(noisy_labels, device=device)
    
    set_seed(seed)
    model = get_resnet18().to(device)
    opt = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)
    sched = optim.lr_scheduler.MultiStepLR(opt, [50, 75], 0.1)
    
    cached_gv_ref = {'step': 0, 'gv': None}
    
    # Phase 1: Initialize (higher error threshold for high noise)
    for ep in range(INIT_EPOCHS):
        train_one_epoch(model, train_loader, opt, sched, clean_t, noisy_t, INIT_LAMBDA, cached_gv_ref)
    
    init_final_error = 1 - evaluate(model, test_loader)
    
    # For η=0.8, ordered threshold is higher (~30%)
    if init_final_error > 0.35:
        print(f'    ⚠️ Init failed: error={init_final_error:.4f}')
        return {
            'seed': seed, 'init_error': init_final_error, 'init_success': False,
            'lifetime': 0, 'censored': False, 'final_error': init_final_error, 'trajectory': []
        }
    
    # Phase 2: Switch to target λ and observe
    trajectory = [{'epoch': 0, 'error': init_final_error}]
    collapse_count = 0
    lifetime = None
    
    for ep in range(1, MAX_OBSERVE_EPOCHS + 1):
        train_one_epoch(model, train_loader, opt, sched, clean_t, noisy_t, TARGET_LAMBDA, cached_gv_ref)
        err = 1 - evaluate(model, test_loader)
        trajectory.append({'epoch': ep, 'error': err})
        
        if err >= COLLAPSE_THRESHOLD:
            collapse_count += 1
            if collapse_count >= COLLAPSE_PERSIST:
                lifetime = ep - COLLAPSE_PERSIST + 1
                break
        else:
            collapse_count = 0
    
    censored = (lifetime is None)
    if censored:
        lifetime = MAX_OBSERVE_EPOCHS
    
    final_error = 1 - evaluate(model, test_loader)
    
    return {
        'seed': seed, 'init_error': init_final_error, 'init_success': True,
        'lifetime': lifetime, 'censored': censored, 'final_error': final_error, 'trajectory': trajectory
    }

In [ ]:
trainset, testset = load_cifar10()
clean_labels = np.array(trainset.targets)
train_loader = DataLoader(IndexedDataset(trainset), BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
test_loader = DataLoader(testset, BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

m = get_resnet18().to(device)
for _ in range(10): _ = m(torch.randn(BATCH_SIZE,3,32,32,device=device))
del m; torch.cuda.empty_cache()
print('Ready')

In [ ]:
results = []
ckpt = f'{SAVE_DIR}/{NOTEBOOK_ID}_checkpoint.json'
done_seeds = set()

if os.path.exists(ckpt):
    results = json.load(open(ckpt))
    done_seeds = {r['seed'] for r in results}
    print(f'Loaded: {len(done_seeds)} done')

for seed in range(N_SEEDS):
    if seed in done_seeds:
        continue
    
    run_num = len(results) + 1
    print(f'\n[{run_num}/{N_SEEDS}] Seed {seed} (η={NOISE_RATE})')
    
    noisy_labels = inject_label_noise(clean_labels, NOISE_RATE, seed)
    
    t0 = time.time()
    result = run_lifetime_experiment(seed, train_loader, test_loader, clean_labels, noisy_labels)
    dt = time.time() - t0
    
    result['experiment_id'] = f'{NOTEBOOK_ID}-{seed:03d}'
    result['target_lambda'] = TARGET_LAMBDA
    result['eta'] = NOISE_RATE
    result['time'] = dt
    
    results.append(result)
    done_seeds.add(seed)
    
    status = '⏱️ CENSORED' if result['censored'] else f"💀 COLLAPSED at {result['lifetime']}"
    print(f'  Init: {result["init_error"]:.4f} | {status} | Final: {result["final_error"]:.4f} | {dt/60:.1f}min')
    
    json.dump(results, open(ckpt, 'w'), indent=2)
    remaining = N_SEEDS - run_num
    print(f'  ETA: {remaining*dt/3600:.1f}h')
    
    torch.cuda.empty_cache()

print('\n' + '='*50 + f'\n{NOTEBOOK_ID} DONE\n' + '='*50)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

json.dump(results, open(f'{SAVE_DIR}/{NOTEBOOK_ID}_results.json', 'w'), indent=2)
df = pd.DataFrame([{k: v for k, v in r.items() if k != 'trajectory'} for r in results])
df.to_csv(f'{SAVE_DIR}/{NOTEBOOK_ID}_results.csv', index=False)

print('='*60)
print(f'{NOTEBOOK_ID} SUMMARY - HIGH NOISE LIFETIME')
print(f'Target λ = {TARGET_LAMBDA}, η = {NOISE_RATE} (HIGH NOISE)')
print('='*60)

successful = df[df['init_success'] == True]
n_total = len(successful)
n_collapsed = (successful['censored'] == False).sum()
n_censored = (successful['censored'] == True).sum()

print(f'\n📊 Results (n={n_total}):')
print(f'   Collapsed: {n_collapsed} ({100*n_collapsed/n_total:.1f}%)')
print(f'   Censored:  {n_censored} ({100*n_censored/n_total:.1f}%)')

if n_total > 0:
    lifetimes = successful['lifetime'].values
    print(f'\n⏱️ Lifetime Statistics:')
    print(f'   Mean: {lifetimes.mean():.1f} epochs')
    print(f'   Median: {np.median(lifetimes):.1f} epochs')

# Init failure rate
n_failed = (df['init_success'] == False).sum()
print(f'\n⚠️ Init failures: {n_failed}/{len(df)} ({100*n_failed/len(df):.1f}%)')

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

successful = df[df['init_success'] == True]
if len(successful) > 0:
    lifetimes = successful['lifetime'].values
    
    ax = axes[0]
    times = np.arange(0, MAX_OBSERVE_EPOCHS + 1)
    survival = np.array([np.mean(lifetimes >= t) for t in times])
    ax.step(times, survival, where='post', linewidth=2, color='purple')
    ax.fill_between(times, survival, step='post', alpha=0.2, color='purple')
    ax.axhline(0.5, color='gray', linestyle='--', alpha=0.7)
    ax.set_xlabel('Time (epochs)', fontsize=12)
    ax.set_ylabel('Survival Probability', fontsize=12)
    ax.set_title(f'Survival Curve (λ={TARGET_LAMBDA}, η={NOISE_RATE})', fontsize=12)
    ax.set_xlim(0, MAX_OBSERVE_EPOCHS)
    ax.set_ylim(0, 1.05)
    ax.grid(True, alpha=0.3)
    
    ax = axes[1]
    collapsed_lifetimes = successful[successful['censored'] == False]['lifetime'].values
    if len(collapsed_lifetimes) > 0:
        ax.hist(collapsed_lifetimes, bins=20, color='purple', alpha=0.7, edgecolor='black')
    ax.axvline(MAX_OBSERVE_EPOCHS, color='blue', linestyle='--', linewidth=2)
    ax.set_xlabel('Lifetime (epochs)', fontsize=12)
    ax.set_ylabel('Count', fontsize=12)
    ax.set_title('Lifetime Distribution (High Noise)', fontsize=12)
    ax.grid(True, alpha=0.3)
    
    ax = axes[2]
    n_show = min(10, len(results))
    for r in results[:n_show]:
        if r['trajectory']:
            epochs = [t['epoch'] for t in r['trajectory']]
            errors = [t['error'] for t in r['trajectory']]
            color = 'red' if not r['censored'] else 'blue'
            ax.plot(epochs, errors, alpha=0.5, color=color, linewidth=1)
    ax.axhline(COLLAPSE_THRESHOLD, color='orange', linestyle='--', linewidth=2)
    ax.axhline(0.30, color='green', linestyle='--', linewidth=2, label='Ordered (high noise)')
    ax.set_xlabel('Epoch', fontsize=12)
    ax.set_ylabel('Test Error', fontsize=12)
    ax.set_title('Sample Trajectories (η=0.8)', fontsize=12)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{SAVE_DIR}/figures/{NOTEBOOK_ID}_survival_analysis.png', dpi=150)
plt.show()

print(f'\n📊 Comparison note:')
print(f'   η=0.4, λ=0.50: transition point')
print(f'   η=0.8, λ=0.44: transition point (shifted due to high noise)')